# Generative Point Cloud Models for Object Detection
To fit point clouds to bounding boxes, we'll develop approaches for sampling bounding boxes that minimize the Chamfer distance between their point-transformed bounding box equivalents and the observed point cloud.  We'll do this using [Gen](https://www.gen.dev/), a novel probabilistic programming language built on top of Julia.

### Import block
If any of the blocks below are missing, please go through the following process below to have them installed:

1. Go to a Julia REPL (if Julia is installed, simply type `julia` in your shell where it is recognized).
2. Type `using Pkg`.
3. Add the name of the missing package with: `Pkg.add("<PKG NAME>")` (without the braces "<" and ">").

In [ ]:
using PyPlot  # Import PyPlot
using Gen  # Import gen
using Random, Distributions  # Synthetic dataset generation
using Plots  # Displaying our data
using PyCall  # Python interop
using LinearAlgebra  # Chamfer distance computation
using Plotly  # Plotting grids

### Create a Synthetic Dataset
We'll use Julia random numbers framework framework to create a function for a synthetic dataset.

In [ ]:
# First, set random seed
rng = MersenneTwister(8080);

#### Uniform PC Model
Samples points uniformly over the boundary $[0,1]^3$.

In [ ]:
function pc_model_uniform(N::Int)
    """ Generative model for our point cloud synthetic dataset. Samples random 3-dimensional points in 
    the domain [0, 1].
    """
    return rand!(rng, zeros(N, 3))
end

#### Bounding Box PC Model
Samples points from the edges of a proposed bounding box and adds noise to them to construct a point cloud.

In [ ]:
function pc_model_bbox_random(N, L, W, H, sigma, xc=0.0, yc=0.0, zc=0.0)
    """ Generative model for our point cloud synthetic dataset. Samples random 3-dimensional points using Gaussian 
    distributions centered on the coordinates of a specified bounding box.
    """
    # Compute points per segment
    N_seg = convert(Int32, N/12)
    
    # Get origin
    x0 = xc - L/2
    y0 = yc - W/2
    z0 = zc - H/2
    
    # First group of segments
    s1 = [[x0+(i*L)/N_seg, y0, z0]+rand(Normal(0, sigma), 3) for i in 1:N_seg]
    s2 = [[x0+(i*L)/N_seg, y0+W, z0]+rand(Normal(0, sigma), 3) for i in 1:N_seg]
    s3 = [[x0+(i*L)/N_seg, y0, z0+H]+rand(Normal(0, sigma), 3) for i in 1:N_seg]
    s4 = [[x0+(i*L)/N_seg, y0+W, z0+H]+rand(Normal(0, sigma), 3) for i in 1:N_seg]
    
    # Second group of segments
    s5 = [[x0, y0+(j*W)/N_seg, z0]+rand(Normal(0, sigma), 3) for j in 1:N_seg]
    s6 = [[x0+L, y0+(j*W)/N_seg, z0]+rand(Normal(0, sigma), 3) for j in 1:N_seg]
    s7 = [[x0, y0+(j*W)/N_seg, z0+H]+rand(Normal(0, sigma), 3) for j in 1:N_seg]
    s8 = [[x0+L, y0+(j*W)/N_seg, z0+H]+rand(Normal(0, sigma), 3) for j in 1:N_seg]
    
    # Third group of segments
    s9 = [[x0, y0, z0+(k*H)/N_seg]+rand(Normal(0, sigma), 3) for k in 1:N_seg]
    s10 = [[x0+L, y0, z0+(k*H)/N_seg]+rand(Normal(0, sigma), 3) for k in 1:N_seg]
    s11 = [[x0, y0+W, z0+(k*H)/N_seg]+rand(Normal(0, sigma), 3) for k in 1:N_seg]
    s12 = [[x0+L, y0+W, z0+(k*H)/N_seg]+rand(Normal(0, sigma), 3) for k in 1:N_seg]
    
    segments = [s1, s2, s3, s4, s5, s6, s7, s8, s9, s10, s11, s12]
    A = zeros(12*N_seg+1, 3)
    A[1,:] = [0 0 0]
    index = 2
    S_tot = zeros(3)
    for s in segments
        for si in s
            A[index,:] = si
            S_tot += si
            index += 1
        end
    end
    mean = S_tot*(1/N)
    return A
end

### Transform Bounding Box to Point Cloud
For visualization and for computing the Chamfer distance between the point cloud and bounding box, we'll need to convert our bounding box to a point cloud as well.  Below is our function to accomplish this.

In [ ]:
function box2points(dims, points_per_seg, xc=0.0, yc=0.0, zc=0.0)
    """Function for transforming a given bounding box (determined by length (L), width (W), and height (H), 
    as well as a center point given by (xc, yc, zc).  The total number of points in this point cloud is equal
    to (12*points_per_seg) + 1."""
    
    # Get dimensions of box
    (L, W, H) = dims
    
    # Get origin
    x0 = xc - L/2
    y0 = yc - W/2
    z0 = zc - H/2
    
    # First group of segments
    s1 = [[x0+((i*L)/points_per_seg), y0, z0] for i in 1:points_per_seg]
    s2 = [[x0+((i*L)/points_per_seg), y0+W, z0] for i in 1:points_per_seg]
    s3 = [[x0+((i*L)/points_per_seg), y0, z0+H] for i in 1:points_per_seg]
    s4 = [[x0+((i*L)/points_per_seg), y0+W, z0+H] for i in 1:points_per_seg]
    
    # Second group of segments
    s5 = [[x0, y0+((j*W)/points_per_seg), z0] for j in 1:points_per_seg]
    s6 = [[x0+L, y0+((j*W)/points_per_seg), z0] for j in 1:points_per_seg]
    s7 = [[x0, y0+((j*W)/points_per_seg), z0+H] for j in 1:points_per_seg]
    s8 = [[x0+L, y0+((j*W)/points_per_seg), z0+H] for j in 1:points_per_seg]
    
    # Third group of segments
    s9 = [[x0, y0, z0+((k*H)/points_per_seg)] for k in 1:points_per_seg]
    s10 = [[x0+L, y0, z0+((k*H)/points_per_seg)] for k in 1:points_per_seg]
    s11 = [[x0, y0+W, z0+((k*H)/points_per_seg)] for k in 1:points_per_seg]
    s12 = [[x0+L, y0+W, z0+((k*H)/points_per_seg)] for k in 1:points_per_seg]
    
    # Concatenate all segments
    segments = [s1, s2, s3, s4, s5, s6, s7, s8, s9, s10, s11, s12]
    
    # Initialize output data structure
    A = zeros(12*points_per_seg+1, 3)
    A[1,:] = [0 0 0]
    index = 2
    for s in segments  # Add segments
        for si in s  # Add points in segments
            A[index,:] = si
            index += 1
        end
    end
    return A
    end    

### Visualize synthetic dataset
We'll use the PyPlot library to qualitatively examine our data, and later, this will help us determine the goodness of fit for our bounding boxes.

In [ ]:
function plot_data(points1, points2, title="")
    """Function for plotting point cloud data, usually comparing a point-transformed bounding box to an 
    observed point cloud.  For visualizing a single point cloud, simply pass the same set of points in twice."""
    
    Plots.scatter(points2[:,1], points2[:,2], points2[:,3], label="Bounding Box", markersize=1, title=title)
    Plots.scatter!(points1[:,1], points1[:,2],points1[:,3], label="Point Cloud", markersize=3)
end
    

#### Sample Plotting
See below for an example with constructing and plotting point clouds.

In [ ]:
box_points = box2points((1, 1, 1), 1000)
pc_points = pc_model_uniform(1000)
plot_data(pc_points, box_points)

### Plot Combined
Function for visualizing several traces/point clouds at once.

In [ ]:
function plot_grid(traces, point_cloud, points_per_seg)
    
    # Iterate through traces
    for (i,trace) in enumerate(traces)
        
        # Get choices from traces
        choices = Gen.get_choices(trace)
        
        # Get parameters
        L1 = choices[:L]
        W1 = choices[:W]
        H1 = choices[:H]
        # Convert to points
        box_points = box2points((L1, W1, H1), points_per_seg)
        
        # Create scatterplots
        plt = Plots.scatter(box_points[:,1],box_points[:,2],box_points[:,3], label="Bounding Box", markersize=1, title= "z =$choices[:z]")
        Plots.scatter!(point_cloud[:,1],point_cloud[:,2],point_cloud[:,3], label="Point Cloud", markersize=2)
        Plots.display(plt)
    end
end

        

### Chamfer distance for scoring proposed bounding boxes
We'll evaluate the likelihood of a bounding box using the Chamfer distance.  We'll choose proposed bounding boxes that minimize the Chamfer distance between the point-transformed point cloud and the observed point cloud.

In [ ]:
# Implementation source: http://graphics.stanford.edu/courses/cs468-17-spring/LectureSlides/L14%20-%203d%20deep%20learning%20on%20point%20cloud%20representation%20(analysis).pdf
function chamfer_distance(points_A, points_B)
    """Function for computing the Chamfer distance between two point clouds ('points_A' and 'points_B').  Note: this metric is
    not normalized by the total number of points in the point cloud."""
    
    # Used to keep track of total chamfer distance
    tot = 0
    
    # Compute error for point cloud A
    for pA in points_A
        X = [pA-pB for pB in points_B]
        tot += minimum([abs2(LinearAlgebra.norm(X[i,:], 2)) for i in 1:convert(Int32, length(X)/3)])
    end
    
    # Compute error for point cloud B
    for pB in points_B
        X = [pB-pA for pA in points_A]
        tot += minimum([abs2(LinearAlgebra.norm(X[i,:], 2)) for i in 1:convert(Int32, length(X)/3)])
    end
    return tot
end
    

### Generative Model using Multivariate Gaussian Normal
To do a Bayesian fitting of a bounding box, we first need to center the bounding box on the point cloud.  To accomplish this, we'll use a generative model that samples a given **L, W, H**.

This bounding box generative model will be created using **Gen**.

In [ ]:
@gen function mv_bounding_box_model(bounds, N, zeta=0.01)
    """Generative model for our bounding box using Gen.  This generative function samples a center point of the 
    point cloud (xc, yc, zc), as well as dimensions length (L), width (W), and height (H).  It also samples a 
    standard deviation sigma.  With all of these variables, it then samples points from the proposed center point 
    according to the multivariate normal distribution N([xc, yc, zc], sigma*I3)."""
    
    # Get bounds from arguments
    ((x_min, x_max), (y_min, y_max), (z_min, z_max), (sigma_min, sigma_max)) = bounds    
    
    # Sample center coordinate of the bounding box
    xc = @trace(uniform(x_min, x_max), :xc)
    yc = @trace(uniform(y_min, y_max), :yc)
    zc = @trace(uniform(z_min, z_max), :zc)
    
    # Sample L, W, H
    W = @trace(uniform(0, 1), :W)
    L = @trace(uniform(0, 1), :L)
    H = @trace(uniform(0, 1), :H)
    
    # Sample standard deviation for the center of the point cloud
    sigma = @trace(uniform(sigma_min, sigma_max), :sigma)
    
    mean = [xc, yc, zc]
    cov = Matrix(sigma*I, 3, 3)
    
    # Now sample N points using a multivariate normal distribution
    sampled_points = zeros(N, 3)
    for i in 1:N
        sampled_points[i, :] = @trace(mvnormal(mean, Matrix(sigma*I, 3, 3)), (:y, i))
    end
    # We also use the returned y values
end
    

### Custom Proposal Distribution
To increase the sample efficiency of our estimates, we can use a custom proposal distribution that computes the mean of the observed dataset, and uses this as the mean $(x_c, y_c, z_c)$ coordinate of the bounding box.

In [ ]:
@gen function mean_pc_proposal(observations, zeta=0.01)
    """Gen proposal function for more efficiently proposing point clouds.  Samples the mean of the proposed point
    cloud at three-dimensional locations that are noisy means of the true location of the observed point cloud.
    Effectively, this ensures that the proposed bounding box is centered around the observed point cloud.  This
    is used as a custom proposal distribution in our importance sampling method."""
    
    # Compute the mean of the observations
    mu = sum(observations, dims=1) / (length(observations) ÷ 3)
    
    # Sample the center point of the point cloud
    xc = @trace(normal(mu[1], zeta), :xc)
    yc = @trace(normal(mu[2], zeta), :yc)
    zc = @trace(normal(mu[3], zeta), :zc)
    
    return nothing
end

### Inference
With our generative models defined, we are now ready to perform inference for bounding boxes based off of datasets.  We'll use importance sampling with our custom proposal function.

In [ ]:
# We can now write the do_inference function
function inference(model, proposal, bounds, ys, amount_of_computation)
    """Function for carrying out inference with our generative model and custom proposal and importance sampling."""

    # Create a choice map for our observed point cloud
    observations = Gen.choicemap()
    N = length(ys) ÷ 3  # Compute length of ys for generative model
    for i in 1:N  # Add observed points to choice map
        observations[(:y, i)] = ys[i,:]
    end
    
    # Call importance_resampling to obtain a likely trace consistent with observations
    (traces, _) = Gen.importance_sampling(model, (bounds, N), observations, proposal, 
                                         (ys,), amount_of_computation);
    return traces
end;

#### Sample Inference Procedure
Here's an example of how we can run inference.

In [ ]:
# Create observations
L = 0.5
W = 0.25
H = 0.1
N_points = 240
epsilon = 0.02
observations = pc_model_bbox_random(N_points, L, W, H, epsilon)

# Number of traces
num_traces = 100

# Now run inference
x_min, x_max, y_min, y_max, z_min, z_max = 0, 1, 0, 1, 0, 1
sigma_min = 0.01
sigma_max = 0.1
bounds = ((x_min, x_max), (y_min, y_max), (z_min, z_max), (sigma_min, sigma_max))
traces = inference(mv_bounding_box_model, mean_pc_proposal, bounds, observations, num_traces);
println("Number of traces: $(length(traces))")

### Compute Z From Traces
Using our traces sampled via importance sampling, which help us to find the center of a bounding box, we can then compute Z, which is a chamfer distance between a proposed bounding box and a point cloud.  We can construct an empirical distribution of z over all these choices, and then condition on z = 0 (the optimal value of z).

In [ ]:
function compute_noisy_chamfer(traces, observed_points)
    """Compute Chamfer distance between proposed traces and and observed points."""
    
    # Get sampled choice values from traces
    choices = [Gen.get_choices(trace) for trace in traces]
    bboxes = [(choice[:L], choice[:W], choice[:H]) for choice in choices]
    
    # Initialize output Chamfer distance values
    zs = zeros(length(traces))
    N_observed_points = length(observed_points) ÷ 3
    
    # Iterate through bounding boxes from traces
    for (i, bbox) in enumerate(bboxes)
        if i % 100 == 0
            println("Iterated through $i traces")
        end
        bbox_points = box2points(bbox, N_observed_points ÷ 12)
        zs[i] = chamfer_distance(bbox_points, observed_points)
    end
    return zs
end

#### Sample Chamfer distance computation and plotting
We can compute the Chamfer distances we sampled above, and then we can use this to plot a histogram and time series of the Chamfer distance values over time.

In [ ]:
zs = compute_noisy_chamfer(traces, observations);

In [ ]:
function plot_hist_and_time_series(zs)
    """Function for plotting the histogram and time series of data over Chamfer distance values for a set of traces."""
    p1 = Plots.histogram(zs, bins=10, title="Z across traces")
    step = 10
    p2 = Plots.plot([i for i in 1:step:length(zs)], [zs[i] for i in 1:step:length(zs)], title="Z vs. number of resampled traces")
    Plots.plot(p1, p2, layout = (1, 2), legend = false)
end

In [ ]:
# Run the function
plot_hist_and_time_series(zs)

### Sample Conditionally on the traces closest to Zero
As the final stage of our pipeline, we can sample from the smallest noisy Chamfer distance values to find candidate proposals that best fit the data.

In [ ]:
sorted_indices = sortperm(zs)

# Now find best bounding box parameters
choices = [Gen.get_choices(trace) for trace in traces]
bboxes = [(choice[:L], choice[:W], choice[:H]) for choice in choices]
centers = [[choice[:xc], choice[:yc], choice[:zc]] for choice in choices]

best_bboxes = [bboxes[index] for index in sorted_indices[1:10]]
best_centers = [centers[index] for index in sorted_indices[1:10]]
#best_zs = [zs[index] for index in sorted_indices][1:10]
bboxes2points = [box2points(best_bboxes[i], 100, best_centers[i][1], best_centers[i][2], best_centers[i][3]) for i in 1:10]

In [ ]:
plot_data(observations, bboxes2points[1])

## Generative Modeling and Inference for Object Detection on A2D2 Data
Now that we have shown a proof of concept with synthetic data, we can now analyze the performance of this with real data taken from the Audi Autonomous Driving Dataset.

### Loading Data

In [ ]:
using NPZ
using JSON

In [ ]:
function get_files(path)
    """Load A2D2 files from a given directory of the A2D2 date sub-directories."""
    
    # Get data for lidar
    lidar_path = string(path, "lidar/cam_front_center/")
    lidar_files = readdir(lidar_path)
    lidar_paths = [string(lidar_path, lidar_file) for lidar_file in lidar_files]
    
    # Get data for lidar
    bbox_path = string(path, "label3D/cam_front_center/")
    bbox_files = readdir(bbox_path)
    bbox_paths = [string(bbox_path, bbox_file) for bbox_file in bbox_files]
    
    return lidar_paths, bbox_paths
end

In [ ]:
function load_data(lidar_path)
    """Load data using a NumPy file reader."""
    
    # Initialize output point clouds
    PC = npzread(lidar_path)["points"]
    return PC            
end        

In [ ]:
using JSON
function load_bbox(bbox_path)
    """Load bounding box from a given bounding box path."""
    pc = nothing
    open(bbox_path, "r") do f
        dicttxt = JSON.parse(f)  # file information to string
        boxes = keys(dicttxt)
        box_points = [dicttxt[box]["3d_points"] for box in boxes]
        box_classes = [dicttxt[box]["class"] for box in boxes]
        for (j, box_class) in enumerate(box_classes)
            if box_class == "Car"
                pc = [convert(Array{Float64,1}, box_points[j][i]) for i in 1:length(box_points[j])]
                break
            end
            
        end
    end
    return pc
end

In [ ]:
function normalize_and_crop(bbox_points)
    """Normalize all points to fall in the interval [0, 1]^3, according to the locations of the bounding box 
    point cloud values."""
    
    # Compute the mean of the observations
    x = getindex.(bbox_points, 1)
    y = getindex.(bbox_points, 2)
    z = getindex.(bbox_points, 3)

    min_x, max_x = minimum(x), maximum(x)
    delta_x = max_x-min_x
    min_y, max_y = minimum(y), maximum(y)
    delta_y = max_y-min_y
    min_z, max_z = minimum(z), maximum(z)
    delta_z = max_z-min_z

    
    x_norm = [(xi-min_x)/(2*delta_x) for xi in x]
    y_norm = [(yi-min_y)/(2*delta_y) for yi in y]
    z_norm = [(zi-min_z)/(2*delta_z) for zi in z]
    
    println(x_norm)
    norm_data = zeros(length(x), 3)
    
    i = 1
    for (x,y,z) in zip(x_norm, y_norm, z_norm)
        norm_data[i,:] = [x,y,z]
        i += 1
    end

    return norm_data
end

### Perform inference for synthetic datasets
We'll randomly generate datasets using a noisy bounding box model.  For each, we'll store the associated goal bounding box for comparing it to our top five traces.

In [ ]:
function synthetic_bounding_boxes(N=10, num_points=240, sigma=0.02)
    """Function for creating synthetic bounding bounding boxes."""
    pcs = [zeros(num_points, 3) for i in 1:N]
    bbox_dimensions = [zeros(3) for i in 1:N]
    for i in 1:N
        L, W, H = rand!(rng, zeros(3))
        bbox_dimensions[i] = [L,W,H] 
        pcs[i] = pc_model_bbox_random(num_points, L, W, H, sigma, 0.5, 0.5, 0.5)
    end
    return pcs, bbox_dimensions
end

In [ ]:
function crop_from_pc(pc, bbox_coords)
    
    cropped_cloud = []
    
    x_min = minimum(getindex.(bbox_coords, 1))
    y_min = minimum(getindex.(bbox_coords, 2))
    z_min = minimum(getindex.(bbox_coords, 3))
    x_max = maximum(getindex.(bbox_coords, 1))
    y_max = maximum(getindex.(bbox_coords, 2))
    z_max = maximum(getindex.(bbox_coords, 3))
    N = N = length(pc) ÷ 3
    for i in 1:N
        point = pc[i,:]
        if point[1] >= x_min && point[1] <= x_max && 
           point[2] >= y_min && point[2] <= y_max &&
           point[3] >= z_min && point[3] <= z_max 
            append!(cropped_cloud, point)
        end
    end
    formatted_cloud = convert(Array{Float64,1}, cropped_cloud)
    return formatted_cloud
end
    

In [ ]:
function top_five_traces(point_cloud, original_bbox, num_traces)
    # Bounds
    x_min, x_max, y_min, y_max, z_min, z_max = 0, 1, 0, 1, 0, 1
    sigma_min = 0.01
    sigma_max = 0.1
    bounds = ((x_min, x_max), (y_min, y_max), (z_min, z_max), (sigma_min, sigma_max))
    
    # Perform inference
    traces = inference(mv_bounding_box_model, mean_pc_proposal, bounds, point_cloud, num_traces);
    
    # Sort according to highest-performing traces
    zs = compute_noisy_chamfer(traces, point_cloud);
    sorted_indices = sortperm(zs)
    
    # Now find best bounding box parameters
    choices = [Gen.get_choices(trace) for trace in traces]
    bboxes = [(choice[:L], choice[:W], choice[:H]) for choice in choices]
    centers = [[choice[:xc], choice[:yc], choice[:zc]] for choice in choices]
    
    # Get best ones
    best_bboxes = [bboxes[index] for index in sorted_indices[1:5]]
    best_centers = [centers[index] for index in sorted_indices[1:5]]
    #best_zs = [zs[index] for index in sorted_indices][1:10]
    bboxes2points = [box2points(best_bboxes[i], 20, best_centers[i][1], best_centers[i][2], best_centers[i][3]) for i in 1:5]
    
    # Get best chamfer values
    best_zs = [zs[index] for index in sorted_indices[1:5]]
    # Now compare performance compared to original box
    volume_bboxes = [best_bboxes[i][1] * best_bboxes[i][2] * best_bboxes[i][3] for i in 1:5]

    # Now compare percent of points inside
    points_in = [evaluate_points_inside(best_bboxes[i], best_centers[i], point_cloud) for i in 1:5]

    # Get volume of original bounding box
    volume_original_bbox = original_bbox[1] * original_bbox[2] * original_bbox[3]
    
return best_bboxes, best_centers, best_zs, volume_bboxes, points_in, volume_original_bbox
end

### Accuracy Metric
We'll define the metric below for evaluating the total fraction of points that fall inside the proposed bounding box, and thus would be captured via a "bounding box detection".

In [ ]:
function evaluate_points_inside(bbox, center, point_cloud)
    """Function for determining the total fraction of points inside a proposed bounding box."""
    
    # Set x, y, z bounds of the bounding box
    x_min = center[1] - bbox[1]/2
    x_max = center[1] + bbox[1]/2
    y_min = center[2] - bbox[2]/2
    y_max = center[2] + bbox[2]/2
    z_min = center[3] - bbox[3]/2
    z_max = center[3] + bbox[3]/2
    
    # Keep track of total points inside proposed bounding box and total points
    tot_in = 0
    tot = 0
    
    # Iterate through all observed points and determine if each one is inside/outside the proposed bounding box
    N = length(point_cloud) ÷ 3
    for i in 1:N
        point = point_cloud[i,:]
        tot += 1
        if point[1] >= x_min && point[1] <= x_max && 
           point[2] >= y_min && point[2] <= y_max &&
           point[3] >= z_min && point[3] <= z_max 
            tot_in += 1
        end
    end
    return tot_in/tot
end        

### Function for Performing Inference on Synthetic Data
We'll define the function below to evaluate the performance of our inference system on synthetically-generated data.

In [ ]:
function evaluate_synthetic_data(synthetic_bounding_boxes, synthetic_point_clouds, num_traces)
    """Evaluate inference results on synthetically-generated data."""
    i = 1
    
    # Iterate through synthetically-generated point cloud datasets.
    for (bbox, points) in zip(synthetic_bounding_boxes, synthetic_point_clouds)
        best_bboxes, best_centers, best_zs, volume_bboxes, points_in, volume_original_bbox = top_five_traces(points, bbox, num_traces)
        println("ITERATION $i")
        println("Best bboxes: $best_bboxes")
        println("Best centers: $best_centers")
        println("Best zs: $best_zs")
        println("Best volume: $volume_bboxes")
        println("Points in: $points_in")
        println("Volume original bbox: $volume_original_bbox")
        println("____________________________________________")
        i += 1
    end
end

In [ ]:
# Generate ten synthetic bounding boxes
pcs, bbox_dimensions = synthetic_bounding_boxes(10, 480, 0.02)
evaluate_synthetic_data(bbox_dimensions, pcs, 1000)

In [ ]:
best_bboxes = [(0.747905, 0.743277, 0.68859), (0.786801, 0.756346, 0.804045), (0.82999, 0.796229, 0.742027), (0.753126, 0.858133, 0.774738), (0.705708, 0.810583, 0.807929)]
best_centers = [[0.504716, 0.486144, 0.512572], [0.495003, 0.503749, 0.495114], [0.504857, 0.489224, 0.494899], [0.500917, 0.506313, 0.504669], [0.495903, 0.498853, 0.511353]]
index = 3
bbox2points = box2points(best_bboxes[index], 40, best_centers[index][1], best_centers[index][2], best_centers[index][3])
og_point_cloud = pcs[1]
plot_data(og_point_cloud, bbox2points)

### Perform Inference on A2D2 Dataset
Now we're ready to try this out on our A2D2 dataset!

In [ ]:
function evaluate_real_data(bounding_boxes, point_clouds)
    """Function for performing inference on our A2D2 dataset."""
    for (bbox, points) in zip(bounding_boxes, point_clouds)
        best_bboxes, best_centers, best_zs, volume_bboxes, points_in, volume_original_bbox = top_five_traces(points, bbox)
    end
end

In [ ]:
path = "/home/ryansander/Documents/6.885/final_project/a2d2/20180807_145028/"
lidar_paths, bbox_paths = get_files(path)
point_clouds = [load_data(lidar_paths[i]) for i in 1:10]
bbox_coords = [load_bbox(bbox_paths[i]) for i in 1:10]
cropped_pcs = [crop_from_pc(point_clouds[i], bbox_coords[i]) for i in 1:10]
println("HERE",cropped_pcs)
normalized_bbox_coordinates = [normalize_and_crop(cropped_pcs) for points in cropped_pcs]
println([length(normalized_box_points[i]) for i in 1:10])
normalized_bounding_boxes = [(0.5, 0.5, 0.5) for i in 1:10];

In [ ]:
evaluate_real_data(normalized_bounding_boxes, normalized_box_points)